## Infos ( à supprimer ensuite)

## Variables à ajouter issues des recherches biblio ( à supprimer ensuite)

Variable(s): 

- Population des pays représentés 
- Variable indiquant si le pays est communiste ou pas 
- Variable indiquant si le pays est stable ou pas 
- Variable indiquant si le pays est émergeant ou pas ( otherwise pauvre , riche à peaufiner par la suite) 
- Variable indiquant si le pays a la culture du sport ou pas. 
- Variable indiquant le PIB par habitant 
- Variable indiquant les financements alloués aux sports (olympiques ou pas)


Il faut créer le train et le test avec des données 2024 , il faut les construire , il faut aussi savoir traiter le fait que cette fois ci la Russie ne sera pas disponible .
Faire indice : pays perturbé 1 avec Russie par exemple ou bien on enlève la Russie  (contexte géopolitique simple, on tah les années où j'ai eu un pb )

But c'est de prédire le nombre de médailles pour chaque pays , indicateur simple 
EXTRA POUR AVOIR 20 : Attribuer un poid aux médailles -> pour potentiellement savoir le nb d'or , argent , bronze

Attention au COVID 19 , influence public , web scrapping sur 30 ans de tous les pays qui ont organisé les jeux 
Attention aux pays pas autorisés à concourir 
Pays en guerre qui envoient moins d'athlètes 
Ukraine 
Palestine 
Iles caiman -> doivent être rattachées à un pays 

JO hiver 
JO été Corrélation 
Apparition des femmes dans les JO ( échantillonnage ) 

Culture du sport : Comment la créer ? 
Budget alloué aux sports (investissement , infrastructures ... ) 
Nb de professionnels 
Nb de licencés 
Nb de sports olympiques disponibles à l'école 

Variable : Pays Communiste ou pas 

Croissance pays économique 

Athlète à partir de 25 ans il n'a plus le niveau : on commence à partir de 2000 , JO Sydney 

Indice dvp des pays à recevoir 

Variable pays organisateur 

Sport qui ont des gens avec un âge bien définie (exemple skate Japonais c'est que des jeunes <= 18 ans)

Dataset coupes mondes de discipline  
    
Variable Note selon l'âge 

Athlète qui sont comptés comme participant à deux pays 

Variable : religion majoritaire pays 

Espérance de vie par pays 
Variable pays dvp ou pas 


Recupérer données JO 2018 et 2021

Variable Poids par Médailles : échelloner les JO de 1 à 10 pour 
Variable Poids par année : 

# Introduction  (Business Understanding) 

## Détermination objectifs métier , analytique du projet 

## Identifier les contraintes , limites et risque du projet

## Chargement des packages si besoin 

In [5]:
%load_ext ipython_memory_usage 

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end


In [ ]:
#%imu_stop

## Importation des librairies 

In [1]:
import pandas as pd 
import numpy as np 

# CODE  

### Chargement de la base de données commune 

In [ ]:
# Code 

In [3]:
df=pd.read_csv('data/data_FLO_without_NaN.csv',sep=',')

In [4]:
df

,Name,Sex,Age,Height,Team,NOC,Games,Year,Season,City,...,Weight,Country_org,Population,PIB/habitant,Score_medaille,Country,Host_country,Participation_own_country,Soviet_past,PIB
0,AALERUD Katrine,F,27.0,175.0,Norway,NOR,Tokyo 2020,2020,Summer,Tokyo,...,74.0,Japan,5195921.0,74400.369777,0,Norway,Japan,0,0,5.792670e+11
1,ABAD Nestor,M,28.0,165.0,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,...,71.0,Japan,46418269.0,25831.582305,0,Spain,Japan,0,0,1.397510e+12
2,ABAGNALE Giovanni,M,26.0,198.0,Italy,ITA,Tokyo 2020,2020,Summer,Tokyo,...,81.0,Japan,60802085.0,29957.804315,0,Italy,Japan,0,0,2.010430e+12
3,ABALDE Alberto,M,26.0,200.0,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,...,88.0,Japan,46418269.0,25831.582305,0,Spain,Japan,0,0,1.397510e+12
4,ABALDE Tamara,F,32.0,192.0,Spain,ESP,Tokyo 2020,2020,Summer,Tokyo,...,88.0,Japan,46418269.0,25831.582305,0,Spain,Japan,0,0,1.397510e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78768,Hamzeh Zarini,M,30.0,198.0,Iran,IRI,2016 Summer,2016,Summer,Rio de Janeiro,...,91.0,Brazil,88550570.0,4388.000000,0,Iran,Brazil,0,0,3.885440e+11
78769,Parviz Zeidvand,M,20.0,172.0,Iran,IRI,2000 Summer,2000,Summer,Sydney,...,66.0,Australia,88550570.0,4388.000000,0,Iran,Australia,0,0,3.885440e+11
78770,Parviz Zeidvand,M,23.0,172.0,Iran,IRI,2004 Summer,2004,Summer,Athina,...,66.0,Greece,88550570.0,4388.000000,0,Iran,Greece,0,0,3.885440e+11
78771,Raya Zin Aldden,F,24.0,176.0,Syria,SYR,2012 Summer,2012,Summer,London,...,65.0,Great Britain,22130000.0,420.000000,0,Syria,United Kingdom,0,0,8.970000e+09


# Data understanding

### Etude des types optimaux (Importer code  Nathan ) 

In [ ]:
# Code à rajouter 

### Chargement et analyse de la mémoire (Importer code Nathan 

In [ ]:
# Code à rajouter 

### Modification des noms pays ... (Code Guillaume)

In [ ]:
# code à rajouter 

## EDA (graph, pivot table ,Matrix confusion...)